In this stage we will prepare the input of the three classifiers starting from the output of the previous stage.

To run this notebook we used the following configuration:
* *Software stack*: LCG 94 (it has spark 2.3.1)
* *Platform*: centos7-gcc7
* *Spark cluster*: Hadalytic

In [1]:
# Check if Spark Session has been created correctly
spark

As first thing we will load the parquet file produced in the previous step

In [3]:
data = spark.read \
            .format("parquet") \
            .load("hdfs://hadalytic/project/ML/data/swan/datasets.parquet")

events = data.count()
print("There are {} events".format(events))

There are 72785 events


We can also have a look at the distribution between classes after the filtering

In [4]:
labels = ['QCD', 'tt', 'W+jets']
counts = data.groupBy('label').count().collect()

qcd_events = 0
tt_events = 0 
wjets_events = 0

print('There are:')
for i in range(3):
    print('\t* {} {} events (frac = {:.3f})'
          .format(
              counts[i][1],
              labels[counts[i].label],
              counts[i][1]*1.0/events
          ))
    if counts[i].label==0:
        qcd_events = counts[i][1]
    elif counts[i].label==1:
        tt_events = counts[i][1] 
    elif counts[i].label==2:
        wjets_events = counts[i][1]

There are:
	* 3594 QCD events (frac = 0.049)
	* 32910 tt events (frac = 0.452)
	* 36281 W+jets events (frac = 0.498)


The dataset is umbalanced, we may need to undersample it.
<br>
## Feature preparation

In the parquet produced in the previous step we have three columns:
1. `hfeatures` containing the 14 High Level Features
2. `lfeature` containing the Low Level Features (list of 801 particles each of them with 19 features)
3. `label` identifying the sample

In [5]:
data.printSchema()

root
 |-- hfeatures: vector (nullable = true)
 |-- label: long (nullable = true)
 |-- lfeatures: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



We can begin by preparing the input for the HLF classifier which simply requires to scale features and encode the label. To use Spark `MinMaxScaler` we need to convert the input into `dense vectors`.

In [6]:
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

vector_dense_udf = udf(lambda r : Vectors.dense(r),VectorUDT())
data = data.withColumn('hfeatures_dense',vector_dense_udf('hfeatures'))

We can now build the pipeline to scale HLF and encode labels

In [7]:
from pyspark.ml import Pipeline
from pyspark.ml.feature import OneHotEncoderEstimator
from pyspark.ml.feature import MinMaxScaler

## One-Hot-Encode
encoder = OneHotEncoderEstimator(inputCols=["label"],
                                 outputCols=["encoded_label"],
                                 dropLast=False)

## Scale feature vector
scaler = MinMaxScaler(inputCol="hfeatures_dense",
                      outputCol="HLF_input")

pipeline = Pipeline(stages=[encoder, scaler])

%time fitted_pipeline = pipeline.fit(data)

CPU times: user 299 ms, sys: 105 ms, total: 403 ms
Wall time: 20.4 s


In [8]:
# Apply the pipeline to data
data = fitted_pipeline.transform(data)

New columns has been created, if we want to drop some of them we can use
```Python 
data = data.drop("col-name") 
```

In [9]:
data.printSchema()

root
 |-- hfeatures: vector (nullable = true)
 |-- label: long (nullable = true)
 |-- lfeatures: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- hfeatures_dense: vector (nullable = true)
 |-- encoded_label: vector (nullable = true)
 |-- HLF_input: vector (nullable = true)



Moving on the particle-sequence classifier we need to sort the particles in each event by decreasing $\Delta R$ distance from the isolated lepton, where $$\Delta R = \sqrt{\Delta \eta^2 + \Delta \phi^2}$$

From the production of the low level features we know that the isolated lepton is the first particle of the list and the 19 features are 
```Python 
features = [
    'Energy', 'Px', 'Py', 'Pz', 'Pt', 'Eta', 'Phi',
    'vtxX', 'vtxY', 'vtxZ', 'ChPFIso', 'GammaPFIso', 'NeuPFIso', 
    'isChHad', 'isNeuHad', 'isGamma', 'isEle', 'isMu', 'Charge'
]
```
therefore we need features 5 ($\eta$) and 6 ($\phi$) to compute $\Delta R$. 

In [10]:
import math

class lepAngularCoordinates():
    """
    This class is used to store the lepton and compute DeltaR 
    from the other particles
    """
    def __init__(self, eta, phi):
        self.Eta = eta
        self.Phi = phi
    
    def DeltaR(self, eta, phi):
        deta = self.Eta - eta
        
        dphi = self.Phi - phi       
        pi = math.pi
        while dphi >  pi: dphi -= 2*pi
        while dphi < -pi: dphi += 2*pi
            
        return math.sqrt(deta*deta + dphi*dphi)

In [11]:
from pyspark.sql.types import ArrayType, DoubleType
from sklearn.preprocessing import StandardScaler


@udf(returnType=ArrayType(ArrayType(DoubleType())))
def transform(particles):
    ## The isolated lepton is the first partiche in the list
    ISOlep = lepAngularCoordinates(particles[0][5], particles[0][6])
    
    ## Sort the particles based on the distance from the isolated lepton
    particles.sort(key = lambda part: ISOlep.DeltaR(part[5], part[6]),
                   reverse=True)
    
    ## Standardize
    particles = StandardScaler().fit_transform(particles).tolist()
    
    return particles

In [12]:
data = data.withColumn('GRU_input', transform('lfeatures'))

In [13]:
data.printSchema()

root
 |-- hfeatures: vector (nullable = true)
 |-- label: long (nullable = true)
 |-- lfeatures: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)
 |-- hfeatures_dense: vector (nullable = true)
 |-- encoded_label: vector (nullable = true)
 |-- HLF_input: vector (nullable = true)
 |-- GRU_input: array (nullable = true)
 |    |-- element: array (containsNull = true)
 |    |    |-- element: double (containsNull = true)



## Undersample the dataset

In [14]:
qcd = data.filter('label=0')
tt = data.filter('label=1')
wjets = data.filter('label=2')

In [15]:
# Create the undersampled dataframes
# False means to sample without repetition
tt = tt.sample(False, qcd_events*1.0/tt_events) 
wjets = wjets.sample(False, qcd_events*1.0/wjets_events)

dataUndersampled = qcd.union(tt).union(wjets)

In [16]:
dataUndersampled.groupBy('label').count().show()

+-----+-----+
|label|count|
+-----+-----+
|    0| 3594|
|    1| 3595|
|    2| 3545|
+-----+-----+



## Shuffle the dataset

Because of how the dataset has been created it is made by "three blocks" obtained with the union of three samples. Therefore we need to shuffle the dataset. We splid this dataset into `train`/`test` and shuffle the train dataset.

In [17]:
from pyspark.sql.functions import rand 
trainUndersampled, testUndersampled = dataUndersampled.randomSplit([0.8, 0.2], seed=42)
trainUndersampled = trainUndersampled.orderBy(rand())

Notice that the whole pipeline will be trigger by the action of saving the parquet file

## Save the parquet file

In [18]:
PATH = "hdfs://hadalytic/project/ML/data/swan/"

trainUndersampled.write.parquet(PATH+'trainUndersampled.parquet',
                                              mode='overwrite')
testUndersampled.write.parquet(PATH+'testUndersampled.parquet',
                                              mode='overwrite')